Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/forecasting-github-dau/auto-ml-forecasting-github-dau.png)

<font color="red" size="5"><strong>!Important!</strong> </br>This notebook is outdated and is not supported by the AutoML Team. Please use the supported version ([link](https://github.com/Azure/azureml-examples/tree/main/sdk/python/jobs/automl-standalone-jobs/automl-forecasting-github-dau)).</font>

# Automated Machine Learning
**Github DAU Forecasting**

## Contents
1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Evaluate](#Evaluate)

## Introduction
This notebook demonstrates demand forecasting for Github Daily Active Users Dataset using AutoML.

AutoML highlights here include using Deep Learning forecasts, Arima, Prophet,  Remote Execution and Remote Inferencing, and working with the `forecast` function. Please also look at the additional forecasting notebooks, which document lagging, rolling windows, forecast quantiles, other ways to use the forecast function, and forecaster deployment.

Make sure you have executed the [configuration](https://github.com/Azure/MachineLearningNotebooks/blob/master/configuration.ipynb) before running this notebook.

Notebook synopsis:

1. Creating an Experiment in an existing Workspace
2. Configuration and remote run of AutoML for a time-series model exploring DNNs
4. Evaluating the fitted model using a rolling test 

## Setup


In [ ]:
import os
import azureml.core
import pandas as pd
import numpy as np
import logging
import warnings

from pandas.tseries.frequencies import to_offset

# Squash warning messages for cleaner output in the notebook
warnings.showwarning = lambda *args, **kwargs: None

from azureml.core import Workspace, Experiment, Dataset
from azureml.train.automl import AutoMLConfig
from matplotlib import pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from azureml.train.estimator import Estimator

This notebook is compatible with Azure ML SDK version 1.35.0 or later.

In [ ]:
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

As part of the setup you have already created a <b>Workspace</b>. To run AutoML, you also need to create an <b>Experiment</b>. An Experiment corresponds to a prediction problem you are trying to solve, while a Run corresponds to a specific approach to the problem.

In [ ]:
ws = Workspace.from_config()

# choose a name for the run history container in the workspace
experiment_name = "github-remote-cpu"

experiment = Experiment(ws, experiment_name)

output = {}
output["Subscription ID"] = ws.subscription_id
output["Workspace"] = ws.name
output["Resource Group"] = ws.resource_group
output["Location"] = ws.location
output["Run History Name"] = experiment_name
output["SDK Version"] = azureml.core.VERSION
pd.set_option("display.max_colwidth", None)
outputDf = pd.DataFrame(data=output, index=[""])
outputDf.T

### Using AmlCompute
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you use `AmlCompute` as your training compute resource.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "github-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_DS12_V2", max_nodes=4
    )
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

## Data
Read Github DAU data from file, and preview data.

Let's set up what we know about the dataset. 

**Target column** is what we want to forecast.

**Time column** is the time axis along which to predict.

**Time series identifier columns** are identified by values of the columns listed `time_series_id_column_names`, for example "store" and "item" if your data has multiple time series of sales, one series for each combination of store and item sold.

**Forecast frequency (freq)** This optional parameter represents the period with which the forecast is desired, for example, daily, weekly, yearly, etc. Use this parameter for the correction of time series containing irregular data points or for padding of short time series. The frequency needs to be a pandas offset alias. Please refer to [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects) for more information.

This dataset has only one time series. Please see the [orange juice notebook](https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/automated-machine-learning/forecasting-orange-juice-sales) for an example of a multi-time series dataset.

In [ ]:
import pandas as pd
from pandas import DataFrame
from pandas import Grouper
from pandas import concat
from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()
plt.figure(figsize=(20, 10))
plt.tight_layout()

plt.subplot(2, 1, 1)
plt.title("Github Daily Active User By Year")
df = pd.read_csv("github_dau_2011-2018_train.csv", parse_dates=True, index_col="date")
test_df = pd.read_csv(
    "github_dau_2011-2018_test.csv", parse_dates=True, index_col="date"
)
plt.plot(df)

plt.subplot(2, 1, 2)
plt.title("Github Daily Active User By Month")
groups = df.groupby(df.index.month)
months = concat([DataFrame(x[1].values) for x in groups], axis=1)
months = DataFrame(months)
months.columns = range(1, 49)
months.boxplot()

plt.show()

In [ ]:
target_column_name = "count"
time_column_name = "date"
time_series_id_column_names = []
freq = "D"  # Daily data

### Split Training data into Train and Validation set and Upload to Datastores

In [ ]:
from helper import split_fraction_by_grain
from helper import split_full_for_forecasting

train, valid = split_full_for_forecasting(df, time_column_name)

# Reset index to create a Tabualr Dataset.
train.reset_index(inplace=True)
valid.reset_index(inplace=True)
test_df.reset_index(inplace=True)

datastore = ws.get_default_datastore()
train_dataset = Dataset.Tabular.register_pandas_dataframe(
    train, target=(datastore, "dataset/"), name="Github_DAU_train"
)
valid_dataset = Dataset.Tabular.register_pandas_dataframe(
    valid, target=(datastore, "dataset/"), name="Github_DAU_valid"
)
test_dataset = Dataset.Tabular.register_pandas_dataframe(
    test_df, target=(datastore, "dataset/"), name="Github_DAU_test"
)

### Setting forecaster maximum horizon 

The forecast horizon is the number of periods into the future that the model should predict. Here, we set the horizon to 14 periods (i.e. 14 days). Notice that this is much shorter than the number of months in the test set; we will need to use a rolling test to evaluate the performance on the whole test set. For more discussion of forecast horizons and guiding principles for setting them, please see the [energy demand notebook](https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/automated-machine-learning/forecasting-energy-demand).  

In [ ]:
forecast_horizon = 14

## Train

Instantiate a AutoMLConfig object. This defines the settings and data used to run the experiment.

|Property|Description|
|-|-|
|**task**|forecasting|
|**primary_metric**|This is the metric that you want to optimize.<br> Forecasting supports the following primary metrics <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>
|**iteration_timeout_minutes**|Time limit in minutes for each iteration.|
|**training_data**|Input dataset, containing both features and label column.|
|**label_column_name**|The name of the label column.|
|**enable_dnn**|Enable Forecasting DNNs|


In [ ]:
from azureml.automl.core.forecasting_parameters import ForecastingParameters

forecasting_parameters = ForecastingParameters(
    time_column_name=time_column_name,
    forecast_horizon=forecast_horizon,
    freq="D",  # Set the forecast frequency to be daily
)

# To only allow the TCNForecaster we set the allowed_models parameter to reflect this.
automl_config = AutoMLConfig(
    task="forecasting",
    primary_metric="normalized_root_mean_squared_error",
    experiment_timeout_hours=1.5,
    training_data=train_dataset,
    label_column_name=target_column_name,
    validation_data=valid_dataset,
    verbosity=logging.INFO,
    compute_target=compute_target,
    max_concurrent_iterations=4,
    max_cores_per_iteration=-1,
    enable_dnn=True,
    allowed_models=["TCNForecaster"],
    forecasting_parameters=forecasting_parameters,
)

We will now run the experiment, starting with 10 iterations of model search. The experiment can be continued for more iterations if more accurate results are required. Validation errors and current status will be shown when setting `show_output=True` and the execution will be synchronous.

In [ ]:
remote_run = experiment.submit(automl_config, show_output=True)

In [ ]:
# If you need to retrieve a run that already started, use the following code
# from azureml.train.automl.run import AutoMLRun
# remote_run = AutoMLRun(experiment = experiment, run_id = '<replace with your run id>')

Displaying the run objects gives you links to the visual tools in the Azure Portal. Go try them!

### Retrieve the Best Model for Each Algorithm
Below we select the best pipeline from our iterations. The get_output method on automl_classifier returns the best run and the fitted model for the last fit invocation. There are overloads on get_output that allow you to retrieve the best run and fitted model for any logged metric or a particular iteration.

In [ ]:
from helper import get_result_df

summary_df = get_result_df(remote_run)
summary_df

In [ ]:
from azureml.core.run import Run
from azureml.widgets import RunDetails

forecast_model = "TCNForecaster"
if not forecast_model in summary_df["run_id"]:
    forecast_model = "ForecastTCN"

best_dnn_run_id = summary_df[summary_df["Score"] == summary_df["Score"].min()][
    "run_id"
][forecast_model]
best_dnn_run = Run(experiment, best_dnn_run_id)

In [ ]:
best_dnn_run.parent
RunDetails(best_dnn_run.parent).show()

In [ ]:
best_dnn_run
RunDetails(best_dnn_run).show()

## Evaluate on Test Data

We now use the best fitted model from the AutoML Run to make forecasts for the test set.  

We always score on the original dataset whose schema matches the training set schema.

In [ ]:
# preview the first 3 rows of the dataset
test_dataset.take(5).to_pandas_dataframe()

In [ ]:
compute_target = ws.compute_targets["github-cluster"]
test_experiment = Experiment(ws, experiment_name + "_test")

In [ ]:
import os
import shutil

script_folder = os.path.join(os.getcwd(), "inference")
os.makedirs(script_folder, exist_ok=True)
shutil.copy("infer.py", script_folder)

In [ ]:
from helper import run_inference

test_run = run_inference(
    test_experiment,
    compute_target,
    script_folder,
    best_dnn_run,
    test_dataset,
    valid_dataset,
    forecast_horizon,
    target_column_name,
    time_column_name,
    freq,
)

In [ ]:
RunDetails(test_run).show()

In [ ]:
from helper import run_multiple_inferences

summary_df = run_multiple_inferences(
    summary_df,
    experiment,
    test_experiment,
    compute_target,
    script_folder,
    test_dataset,
    valid_dataset,
    forecast_horizon,
    target_column_name,
    time_column_name,
    freq,
)

In [ ]:
for run_name, run_summary in summary_df.iterrows():
    print(run_name)
    print(run_summary)
    run_id = run_summary.run_id
    test_run_id = run_summary.test_run_id
    test_run = Run(test_experiment, test_run_id)
    test_run.wait_for_completion()
    test_score = test_run.get_metrics()[run_summary.primary_metric]
    summary_df.loc[summary_df.run_id == run_id, "Test Score"] = test_score
    print("Test Score: ", test_score)

In [ ]:
summary_df